In [1]:
import sys
#BASE_DIR = "/home/dzigen/Desktop/ITMO/smiles2024/RAG-project-SMILES-2024-"
BASE_DIR = "/trinity/home/team06/workspace/mikhail_workspace/rag_project"
sys.path.insert(0, BASE_DIR)

import pandas as pd 
import ast
import numpy as np
import json
from tqdm import tqdm
from IPython.display import clear_output
import os

from src.Retriever import RerankRetriever
from src.Scorer import SimilarityScorerConfig
from src.Reader import LLM_Model
from src.utils import ReaderMetrics, RetrieverMetrics, save_rag_trial_log, prepare_rerankretriever_configs, prepare_reader_configs, load_benchmarks_df
from src.utils import evaluate_retriever, evaluate_reader

/trinity/home/team06/workspace/mikhail_workspace/mikhail_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# !!! TO CHANGE !!!
TRIAL = 1
BENCHMARKS_MAXSIZE = 500
BENCHMARKS_INFO = {'mtssquad': {'db': 'v2', 'table': 'v1'}}

# Retriever part
RETRIEVER_PARAMS = {
    "stage1_retriever_config": {
        "model_path": f"{BASE_DIR}/models/intfloat/multilingual-e5-small",
        "densedb_kwargs": {'metadata': {"hnsw:space": "ip"}},
        "model_kwargs": {'device':'cuda'},
        "encode_kwargs": {'normalize_embeddings': True, 'prompt': 'query: '},
        "params": {'fetch_k': 50, 'threshold': 0.23, 'max_k': 10}},
    "scorer_config": {
        "system_prompt": '''Ты - экспертная система, задача которой - оценить релевантность контекста по отношению к данному вопросу. На вход поступает вопрос и контекст в текстовом формате, твоя задача - вернуть одну из двух меток: "{cert_label}" - в контексте содержится информация, которая может быть использована для верного ответа на данный вопрос и "{uncert_label}" - в контексте не содержится информации, которая может быть использована для верного ответа на данный вопрос. Не пиши ничего другого, кроме предсказываемой метки релевантности: "{cert_label}" либо "{uncert_label}". 

Примеры: 

Вопрос: По просьбе Мадонны поменяли аранжировщика на более опытного?
Контекст: [' Everybody , как и хотела Мадонна, выпускают синглом. При нулевом бюджете на раскрутку фото певицы решают не помещать на обложке, чтобы не отпугнуть цветную аудиторию якобы негритянской диско-соул-певицы . Everybody поднимается на 3-е место в чарте Hot Dance Club Songs, а потом на 107 место в основном, немного не дотянув до первой сотни Hot 100 журнала Billboard[91]. Менеджмент считает это отличным результатом, учитывая нулевые затраты на пиар, и хочет убедиться, что взлёт Everybody не случаен. По просьбе Мадонны вместо Каминса берут более опытного штатного аранжировщика Warner Bros. Records Регги Лукаса (англ.)русск.. Второй сингл Burning Up тоже достигает в чарте танцевальных хитов 3-го места, повторив успех Everybody . И только после этого Мадонне позволяют арендовать студию для записи первого альбома[91].']
Метка: {cert_label}

Вопрос: Для чего осуществляется деятельность официальных органов по получению и применению финансовых средств?
Контекст: [' Общественные финансы связываются с деятельностью государства (или местной власти), заключающейся в получении и применении средств, необходимых для выполнения надлежащих функций[51], с процессом и механизмом формирования и использования государственных ресурсов, балансом доходов и расходов, соответствующим административным контролем[3]. Некоторые авторы[52] также подчёркивают, что предмет общественных финансов лежит на границе между экономикой и политикой, а также отмечают, что в современных обществах доходы и расходы органов государственной власти состоят, почти исключительно, из денежных поступлений.']
Метка: {cert_label}

Вопрос: Кто запретил Нансену высадиться на побережье Гренландии?
Контекст: ['29 февраля 1892 года столетие со дня рождения Россини было отмечено должным образом во Флоренции, но горожане, любившие его музыку, ощущали неловкость оттого, что на могиле Россини в Санта-Кроче все ещё не было подходящего памятника. На первом открытом конкурсе, состоявшемся в 1897 году, комиссия не сочла ни один проект приемлемым. На втором конкурсе, который состоялся в следующем году, новая комиссия одобрила проект Джузеппе Кассиоли. Получившийся в результате монумент был открыт 23 июня 1902 года. Поставленный на фоне северной стены Санта-Кроче, он содержит надпись: Джоаккино Россини . У его основания три датированных картуша гласят: Пезаро , Флоренция , Париж , таким образом, официально отрицая справедливые притязания Болоньи на то, что она внесла свою значительную лепту в длительную славу Россини.']
Метка: {uncert_label}

Вопрос: Кто в 1536 году был губернатором провинции Санта-Марта?
Контекст: ['9 (21) мая владыка Иннокентий заложил в Усть-Зейской станице храм в честь Благовещения Пресвятой Богородицы (по имени этого храма был впоследствии назван город). 12 (24) мая начались переговоры между представителями Цинской империи и чиновниками российского внешнеполитического ведомства во главе с Н. Н. Муравьёвым. Переговоры велись 5 дней, сопровождаясь многочисленными церемониями по китайским и русским обычаям. 16 (28) мая 1858 года был подписан Айгунский договор, по условиям которого весь левый берег Амура признавался российским.']
Метка: {uncert_label}''',
    "user_prompt_template": "Вопрос: {q}\nДокумент: {c}\nМетка: ",
    "cert_label": "1",
    "uncert_label": "0",
    "gen": {'max_new_tokens': 20, 'eos_token_id': 79097},
    "batch_size": 8
    }
}


# Reader part
READER_PARAMS = {
    'prompts': {
        "assistant": 'Отвечай на вопросы, используя информацию из текстов в списке ниже. Если ты не уверена в релевантности данных текстов по отношению к заданному вопросу, то сгенерируй следующий ответ: "У меня нет ответа на ваш вопрос.".',
        "system": "Ты вопросно-ответная система. Все ответы генерируй на русском языке. По вопросам отвечай кратко, чётко и конкретно. Не генерируй излишнюю информацию.",
    },
    'gen': {'max_new_tokens': 512, 'eos_token_id': 79097},
    'data_operate': {'batch_size': 1}
    }


ADDITIONAL_PARAMS = {
    'topk_score_list': 3
}

BERTSCORE_MODEL_PATH = "ru_electra_medium"
# !!! TO CHANGE !!!

SAVE_LOGDIR = f'./logs/trial{TRIAL}'
SAVE_HYPERPARAMS = f'{SAVE_LOGDIR}/hyperparams.json'
SAVE_READERCACHE = f'{SAVE_LOGDIR}/reader_cache.json'
SAVE_RETRIEVERCACHE = f'{SAVE_LOGDIR}/retriever_cache.json'

##### Configure Reader-part

In [3]:
reader_config = prepare_reader_configs(READER_PARAMS)
READER = LLM_Model(reader_config)

Loading checkpoint shards: 100%|██████████| 4/4 [03:41<00:00, 55.28s/it]


In [4]:
sim_score_config = SimilarityScorerConfig()

In [5]:
reader_metrics = ReaderMetrics(BASE_DIR, BERTSCORE_MODEL_PATH, sim_score_config, READER)

Loading Meteor...
Loading ExactMatch


##### Configure Retriever-part

In [6]:
retrievers_config, benchmarks_path = prepare_rerankretriever_configs(BASE_DIR, BENCHMARKS_INFO, RETRIEVER_PARAMS)
retriever_metrics = RetrieverMetrics()

In [7]:
benchmarks_df = load_benchmarks_df(benchmarks_path, BENCHMARKS_MAXSIZE)

In [8]:
RETRIEVERS = {name: RerankRetriever(config, READER) for name, config in retrievers_config.items()}

No sentence-transformers model found with name /trinity/home/team06/workspace/mikhail_workspace/rag_project/models/intfloat/multilingual-e5-small. Creating a new one with mean pooling.


##### Evaluating pipeline

In [9]:
retriever_scores, retriever_cache, predicted_chunks, cache_relevant_flags = evaluate_retriever(
    benchmarks_df, RETRIEVERS, retriever_metrics, show_step=5, topk_score_list=ADDITIONAL_PARAMS['topk_score_list'])

mtssquad


  0%|          | 0/500 [00:00<?, ?it/s]

uncert raw len:  1
uncert raw len:  1
uncert raw len:  1
uncert raw len:  1
uncert raw len:  1
uncert raw len:  1
uncert raw len:  1
uncert raw len:  1


/trinity/home/team06/workspace/mikhail_workspace/rag_project/src/utils/evaluation_metrics.py:42: RuntimeWarning: invalid value encountered in scalar divide
  return (2 * self.precision(pred_cands, gold_cands, k) * self.recall(pred_cands, gold_cands, k)) / (self.precision(pred_cands, gold_cands, k) + self.recall(pred_cands, gold_cands, k))
/trinity/home/team06/workspace/mikhail_workspace/mikhail_venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/trinity/home/team06/workspace/mikhail_workspace/mikhail_venv/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
  0%|          | 1/500 [00:16<2:13:24, 16.04s/it, MRR=0, mAP=0, Recall=0, Precision=0, F1=0, NoRelContextScore=nan]

uncert raw len:  1
uncert raw len:  1
10 0
uncert raw len:  1
uncert raw len:  1
uncert raw len:  1
uncert raw len:  1
uncert raw len:  1
uncert raw len:  1
uncert raw len:  1
uncert raw len:  1


...

uncert raw len:  1
uncert raw len:  1
10 0
uncert raw len:  1
uncert raw len:  1
uncert raw len:  1
uncert raw len:  1
uncert raw len:  1
uncert raw len:  1
uncert raw len:  1
uncert raw len:  1


 19%|█▉        | 97/500 [05:18<20:25,  3.04s/it, MRR=0, mAP=0, Recall=0, Precision=0, F1=0, NoRelContextScore=nan]

uncert raw len:  1
uncert raw len:  1
10 0


 19%|█▉        | 97/500 [05:20<22:13,  3.31s/it, MRR=0, mAP=0, Recall=0, Precision=0, F1=0, NoRelContextScore=nan]


KeyboardInterrupt: 

In [ ]:
contexts = {name: [reader_config.prompts.assistant + "\n\n" + "\n\n".join([f'{i+1}. {doc[1]}' for i, doc in enumerate(docs)]) 
                                                                      for docs in chunks] for name, chunks in predicted_chunks.items()}

In [13]:
reader_scores, reader_cache = evaluate_reader(benchmarks_df, READER, reader_metrics, contexts)

  0%|          | 0/1000 [00:00<?, ?it/s]/trinity/home/team06/workspace/mikhail_workspace/mikhail_venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/trinity/home/team06/workspace/mikhail_workspace/mikhail_venv/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 1000/1000 [1:01:16<00:00,  3.68s/it, BLEU2=0.206, BLEU1=0.257, ExactMatch=0.013, METEOR=0.403, BertScore=nan] 


In [14]:
save_rag_trial_log(SAVE_LOGDIR, reader_scores, retriever_scores, 
                   SAVE_HYPERPARAMS, SAVE_READERCACHE, SAVE_RETRIEVERCACHE,
                   reader_cache, retriever_cache_ids, BENCHMARKS_INFO, BENCHMARKS_MAXSIZE,
                   READER_PARAMS, RETRIEVER_PARAMS)